We will be working in the iPython / Jupyter notebook system. I like these because they are a form of literate programming in which we can mix textbook instruction and explanations with code that can also be run and edited.

The text and mathematics in the notebooks requires a little preliminary learning.

### Markdown

You can document your iPython notebooks by making some cells into **Markdown** cells. Markdown is a way of formatting text that is supposed to be almost as readable un-rendered as when it is tidied up. You might argue that it looks equally bad either way, but that's tough because the notebooks use it and that's how I want you to produce nice looking output to hand in as an assignment !

If you look at the **Markdown** cells as source code (by double-clicking on them) you will see how the raw text looks.

### Maths

In a browser, you can render beautiful equations using a javascript tool called **Mathjax** which is build into the iPython notebooks. 


You can build in symbols to your text such as $\pi$ and $\epsilon$ if you use the \$ signs to indicate where your equations begin and end, and you know enough $\LaTeX$ [try it here !](http://www.codecogs.com/latex/eqneditor.php) to get by.


Equations in 'display' mode are written like this (again look at the source for this cell to see what is used)

\\[ e^{i\pi} + 1 = 0 \\]

or even like this

\begin{equation}
%%
    \nabla^4 \psi = \frac{\partial T}{\partial x}
%%    
\end{equation}

Go back to the rendered form of the cell by 'running' it.

### Links 

[Markdown Website](http://daringfireball.net/projects/markdown/)

[Mathjax Website](http://docs.mathjax.org)

### Dependencies

The notebooks require you to install `cartopy` and `gdal` -- this should be possible with pip but it's not always trivial to do. 


### Datasets

We need to download some data for the examples. Some files are bundled with the notebooks because I am not smart enough to understand how to download them through the various web / ftp interfaces directly or I needed to filter or compress the information for the purposes of the class. Please go to the original sites if you need to use these data for anything other than the demonstrations in these notebooks.

Most datasets are too large for the repository (and generally, that's not a place to keep anything which is not the primary target of revision management. The bundled data are compressed and will be unpacked (copied) to the `Resources` directory by the "download" functions in this notebook. If you mess them up, just run the download again. Anything that is undamaged will just be be skipped anyway.

#### Global Magnetic Data

Magnetic intensity data from [geomag.org](http://geomag.org/models/EMAG2/EMAG2_V2.tif)

#### Topography data

ETOPO1 images are from NOAA - we use their geotifs which are subsampled from the original (enormous) dataset but 

#### NASA Blue marble images

Winter and Summer images for the Earth are grabbed for plotting examples. The winter ones (June) are used by default as these have less ice in the N. Hemisphere. 

#### Earthquake hypocentres

Are grabbed from the [NEIC](http://earthquake.usgs.gov/earthquakes/search/) - they are in the geoJSON format since that is very simple to read with python. The downloads are limited at 20k events so the time and magnitude range is whatever it takes to get just under this limit. The filenames give clues about that, but, so does the catalogue itself once it is read in.

#### Global age grid 

Taken from Earthbyte and reduced in size by throwing away the grid information and saving in compressed numpy format. 

#### Global strain rate

I grabbed the second invariant of the strain rate from the [global strain rate map](http://gsrm.unavco.org/intro) project through the 'Jules Vernes' portal. 



In [ ]:
## Here are some functions to download or install files. 


import requests
import os
import time
import sys
import shutil

def download_file(url, local_filename, expected_size):
    
    
# We might want to bundle some files if they are small / compressed or not readily available for download

    if (os.path.isfile(url)):
        shutil.copy(url, local_filename)
    
    else:
        r = requests.get(url, stream=True)

        start_time = time.time()
        last_time = start_time
        datasize = 0

        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=10000000): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    f.flush()
                    datasize += len(chunk)
                    if (time.time() - last_time) > 2.5:
                        print "{:.1f} Mb in {:.1f}s / {}".format(datasize / 1.0e6, time.time() - start_time, expected_size)
                        last_time = time.time()

    return 

import hashlib

def md5sum(filename, blocksize=65536):
    hash = hashlib.md5()
    with open(filename, "r+b") as f:
        for block in iter(lambda: f.read(blocksize), ""):
            hash.update(block)
    return hash.hexdigest()


def download_cached_file(location_url, local_file, expected_md5, expected_size="Unknown"):

    try:
        assert md5sum(local_file) == expected_md5
        print "Using cached file - {}".format(local_file)
        return(2)
    
    except (IOError, AssertionError) as error_info:
        # No file or the wrong file ... best go download it
        # print "Assertion failed - {}".format(sys.exc_info())
        
        try:
            mag_data_file = download_file(location_url, local_file, expected_size)
            print "Downloaded from {}".format( location_url )
            return(1)

        except:
            print "Unable to download {} [{}] ".format( location_url, sys.exc_info() )
            return(0)



In [ ]:
# Resources required for the tutorial
# The checksum is to make sure any cached file is the right one.
# If the checksum is wrong the file gets downloaded again.

# Some of these files are local, stored in the repo in compressed form and installed into the
# Resources directory. 

resource_list = [
# Global magnetic data 
{
 "local_file":"Resources/EMAG2_image_V2.tif",
 "md5":"20024c06fdabfa57e4f9efb4aabec198",
 "url":"http://geomag.org/models/EMAG2/EMAG2_image_V2.tif",
 "expected_size":"133Mb"
},
    
# Australian Mag Data from AuScope portal (built by hand)
{
 "local_file":"Resources/AusMagAll.tiff.zip",
 "md5":'',
 "url":"Reference/AusMagAll.tiff.zip",
 "expected_size":"10Mb"
},
    
# Blue Marble Image Geotiff June 2004 (11Mb)  
{
 "local_file": "Resources/BlueMarbleNG-TB_2004-06-01_rgb_3600x1800.TIFF",
 "md5": '55e1399674d43a26353d84c114d7ff80',
 "url":"http://neo.sci.gsfc.nasa.gov/servlet/RenderData?si=526294&cs=rgb&format=TIFF&width=3600&height=1800",
 "expected_size":"11Mb"
},
    
    
# Blue Marble Image with Bathymetry png - December 2004 
{
 "local_file": "Resources/BlueMarbleNG-TB_2004-12-01_rgb_3600x1800.TIFF",
 "md5": '825da4b417ae19e24d2ea6db6cf8ad21',
 "url":"http://neo.sci.gsfc.nasa.gov/servlet/RenderData?si=526311&cs=rgb&format=TIFF&width=3600&height=1800",
 "expected_size":"11Mb"
},
    
# Etopo1 - color image 
{
 "local_file": "Resources/color_etopo1_ice_low.tif.zip",
 "md5": '3f53d72e85393deb28874db0c76fbfcb',
 "url":"https://www.ngdc.noaa.gov/mgg/global/relief/ETOPO1/image/color_etopo1_ice_low.tif.zip",
 "expected_size":"35Mb"
},
    
# Etopo1 - bw shaded relief image 
{
 "local_file": "Resources/etopo1_grayscale_hillshade.tif.zip",
 "md5": 'f38b8dc6cd971d72e77edaa837d5b85c',
 "url":"https://www.ngdc.noaa.gov/mgg/global/relief/ETOPO1/image/etopo1_grayscale_hillshade.tif.zip",
 "expected_size":"115Mb"
},
        
# Global age grid  [LOCAL]
{
 "local_file":"Resources/global_age_data.3.6.z.npz",
 "md5":'',
 "url":"Reference/global_age_data.3.6.z.npz",
 "expected_size":"22Mb"
},
      
# Global second invariant of strain rate [LOCAL]
{
 "local_file":"Resources/sec_invariant_strain_0.2.dat.zip",
 "md5":'',
 "url":"Reference/sec_invariant_strain_0.2.dat.zip",
 "expected_size":"4Mb"
},
  
    
# Earthquake data for various regions as json files [LOCAL]
{
 "local_file":"Resources/Earthquake_data.zip",
 "md5":'',
 "url":"Reference/Earthquake_data.zip",
 "expected_size":"4Mb"
},
  
    

    
       
]



In [ ]:

for resource in resource_list:
    print "\nDownloading {:s}".format(resource["local_file"])
    download_cached_file(resource["url"], resource["local_file"], resource["md5"], resource["expected_size"] )


In [ ]:
# Extract any files that were downloaded as zip archives

import zipfile
import glob

for zipped in glob.glob("Resources/*.zip"):
    with zipfile.ZipFile(zipped) as zf:
        zf.extractall("Resources")
        print "Unzipped {}".format(zipped)

In [ ]:
md5sum("Resources/etopo1_grayscale_hillshade.tif.zip") # if you add a new file, add the md5 checksum as well !

In [ ]:
%ls -l Resources/